In [26]:
# ! pip install -q openai
# ! pip install pypdf

In [13]:
from openai import OpenAI
from pypdf import PdfReader

In [14]:
reader = PdfReader("deeplog.pdf")

In [15]:
client = OpenAI(api_key='api-key')

In [16]:
texts = []

In [17]:
for page in reader.pages:
    text = page.extract_text() + "\n"
    text = f"generate 20 questions and their answers from this text without numbers in the beginning and also properly labelling with Prompt and Response without special characters: {text}"
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a researcher"},
      {"role": "user", "content": text}
    ]
    )
    outputs = completion.choices[0].message.content.split('\n')

    print(outputs)

    prompt = None
    for i in range(len(outputs)):

        if not outputs[i].strip():
            continue
        if outputs[i].startswith('Prompt'):
          if prompt is not None:
            raise ValueError("No way")
          prompt = outputs[i].replace('Prompt', '### Human')

        if outputs[i].startswith('Response'):
          if prompt is None:
            raise ValueError("No way")
          response = outputs[i].replace('Response', '### Assistant')
          texts.append(f'{prompt} {response}')
          prompt = None

['Prompt: What is the primary purpose of a system log?', 'Response: The primary purpose of a system log is to record system states and significant events at various critical points to help debug system failures and perform root cause analysis.', '', 'Prompt: What is the importance of log data in computer systems?', 'Response: Log data is an important and valuable resource for understanding system status and performance issues, making system logs naturally excellent sources of information for online monitoring and anomaly detection.', '', 'Prompt: What is DeepLog, and how does it utilize deep learning?', 'Response: DeepLog is a deep neural network model that utilizes Long Short-Term Memory (LSTM) to model a system log as a natural language sequence. This enables DeepLog to automatically learn log patterns from normal execution and detect anomalies when patterns deviate from the model trained from normal log data.', '', 'Prompt: How does DeepLog update its model over time?', 'Response: D

In [21]:
import pandas as pd

In [22]:
df = pd.DataFrame({'texts': texts})

In [24]:
df.head()

,texts
0,### Human: What is the primary purpose of a sy...
1,### Human: What is the importance of log data ...
2,"### Human: What is DeepLog, and how does it ut..."
3,### Human: How does DeepLog update its model o...
4,### Human: What is the significance of constru...


In [25]:
df.to_csv('deeplog_qa.csv', index=False)